# 🧵➡️🔀 Módulo 9 — Multiprocesamiento en Python

En este notebook aprenderás a ejecutar tareas **en paralelo real**, aprovechando múltiples núcleos.

Python tiene dos modelos:
- **Threads** → buena concurrencia, pero limitados por el GIL
- **Procesos (`multiprocessing`)** → paralelismo real (sin GIL)

---

# 1️⃣ Threads vs Procesos

### 🧵 Threads
- Comparten memoria
- Afectados por el **GIL** → no aceleran CPU-bound
- Perfectos para IO-bound

### 🔀 Procesos (`multiprocessing`)
- NO comparten memoria (cada proceso tiene la suya)
- Sin GIL → **paralelismo real**
- Perfectos para **CPU-bound** (cálculo intensivo)

---

# 2️⃣ Crear un proceso básico


In [ ]:
from multiprocessing import Process
import time

def tarea():
    print('Proceso iniciado')
    time.sleep(1)
    print('Proceso terminado')

p = Process(target=tarea)
p.start()
p.join()
print('Fin del programa principal')

---
# 3️⃣ Varios procesos en paralelo


In [ ]:
def trabajo(i):
    print(f'Proceso {i} trabajando...')
    time.sleep(0.5)
    print(f'Proceso {i} terminado')

procesos = []
for i in range(4):
    p = Process(target=trabajo, args=(i,))
    p.start()
    procesos.append(p)

for p in procesos:
    p.join()

---
# 4️⃣ Memoria compartida: `Value` y `Array`

Los procesos NO comparten memoria por defecto.

`Value` → una variable compartida

`Array` → array compartido entre procesos


In [ ]:
from multiprocessing import Value, Array

contador = Value('i', 0)         # entero
lista = Array('i', [1,2,3])      # lista de enteros

def modificar(contador, lista):
    contador.value += 1
    lista[0] = 99

p = Process(target=modificar, args=(contador, lista))
p.start(); p.join()

contador.value, list(lista)

---
# 5️⃣ Pool de procesos (más cómodo y escalable)

El objeto `Pool` administra varios procesos por ti.


In [ ]:
from multiprocessing import Pool

def cuadrado(x):
    return x * x

with Pool(processes=4) as pool:
    resultados = pool.map(cuadrado, range(10))

resultados

---
# 6️⃣ Ejemplo CPU-bound (donde multiprocessing acelera)

Cálculo intensivo: contar primos (lento a propósito).

In [ ]:
def es_primo(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

numeros = list(range(10_000, 10_300))

with Pool(4) as p:
    primos = p.map(es_primo, numeros)

sum(primos)

➡️ Esto sí escala con múltiples núcleos (a diferencia de los threads).

---

# 7️⃣ Comunicación entre procesos: `Queue`


In [ ]:
from multiprocessing import Queue

def producer(q):
    for i in range(5):
        q.put(i)

def consumer(q):
    while not q.empty():
        print('Recibido:', q.get())

q = Queue()
p1 = Process(target=producer, args=(q,))
p2 = Process(target=consumer, args=(q,))
p1.start(); p1.join()
p2.start(); p2.join()

---
# 8️⃣ Ejercicio práctico — Procesamiento paralelo

### 🧩 Objetivos
1. Crear una función que calcule el factorial de un número
2. Usar `Pool.map` para calcular factoriales de 10 números
3. Crear procesos que escriban en un array compartido
4. Crear una `Queue` que recoja mensajes de varios procesos

Escribe tu código abajo:

In [ ]:
# TU CÓDIGO AQUÍ


---
# ✅ Solución (oculta)

<details>
<summary>Mostrar solución</summary>

### 🔹 Factoriales
```python
def factorial(n):
    r = 1
    for i in range(1, n+1):
        r *= i
    return r

with Pool(4) as p:
    res = p.map(factorial, range(1,11))
res
```

### 🔹 Array compartido
```python
arr = Array('i', 10)

def escribir(i, arr):
    arr[i] = i*i

procesos = []
for i in range(10):
    p = Process(target=escribir, args=(i, arr))
    p.start(); procesos.append(p)

for p in procesos: p.join()
list(arr)
```

### 🔹 Queue de mensajes
```python
q = Queue()

def worker(i, q):
    q.put(f"Proceso {i} listo")

procs = []
for i in range(5):
    p = Process(target=worker, args=(i, q))
    p.start(); procs.append(p)

for p in procs: p.join()

while not q.empty():
    print(q.get())
```
</details>